In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [0]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [0]:
from google.colab import drive
drive.mount('drive')

In [0]:
votes = pd.read_csv("drive/My Drive/sna9/train3_data.csv")

In [0]:
def noext(se, mse, y1, y2):
    for i in range(len(votes)):
        if votes[val][i] == votes.itemId[i].min():
                if votes[se][i] == mse[se][k]: y1.append(i)
                else:  y2.append(i)   
    return y1, y2

In [0]:
votes.groupby('itemId')[['Xmin', 'Ymin', 'Xmax', 'Ymax']].describe()

In [0]:
votes.groupby('itemId')[['Xmin', 'Ymin', 'Xmax', 'Ymax']].mean()

In [0]:
votes[['Xmin_true', 'Ymin_true', 'Xmax_true', 'Ymax_true']].describe()

In [0]:
!unzip -qq "drive/My Drive/crowdsourcing_NIatK4b.zip"

In [0]:
answers = pd.read_csv("train_answers.csv")

# Different regressors


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor, VotingRegressor, AdaBoostRegressor, RandomForestRegressor


In [0]:
Xmin_X = votes.groupby('itemId')[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']].median()
Xmin_Y = votes.groupby('itemId')['Xmin_true'].median()

In [0]:
#Xmin_X = votes.groupby('itemId')[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean()
#Xmin_Y = votes.groupby('itemId')['Xmin_true'].mean()
Ymin_X = votes.groupby('itemId')[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean()
Ymin_Y = votes.groupby('itemId')['Ymin_true'].mean()

In [0]:
X_train, X_test, y_train, y_test = train_test_split(Ymin_X, Ymin_Y, test_size=0.2, random_state=42)

In [18]:
clf = GradientBoostingRegressor().fit(X_train, y_train)  
clf.score(X_test, y_test )

0.947106186460356

In [19]:
clf = RandomForestRegressor().fit(X_train, y_train)  
clf.score(X_test, y_test )

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9464848625269798

In [20]:
clf = VotingRegressor([('xgb', GradientBoostingRegressor()), ('rf', RandomForestRegressor())]).fit(X_train, y_train)  
clf.score(X_test, y_test )

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9479348710304041

In [21]:
clf = BaggingRegressor(GradientBoostingRegressor()).fit(X_train, y_train)  
clf.score(X_test, y_test ) # 0.951029407184527 the BEST

0.951029407184527

In [22]:
clf = AdaBoostRegressor(GradientBoostingRegressor()).fit(X_train, y_train)  
clf.score(X_test, y_test )

0.9437872473355248

хмм... от средней лучше, чем от медианной

In [0]:
"""def xgb(X, Y):
    #X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    clf = GradientBoostingRegressor(loss='huber', n_estimators=200, min_samples_leaf=3).fit(X, Y) #_train, y_train)  
    #print(clf.score(X_test, y_test))
    return clf """

def Clf(X, Y, clf):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    clf_fit = clf.fit(X_train, y_train)  
    #print(clf, clf_fit.score(X_test, y_test))
    return clf_fit

In [0]:
clfs = [GradientBoostingRegressor(), RandomForestRegressor(), VotingRegressor([('xgb', GradientBoostingRegressor()), ('rf', RandomForestRegressor())]), 
        BaggingRegressor(GradientBoostingRegressor()), AdaBoostRegressor(GradientBoostingRegressor())]

### ОЧЕНЬ.СТРАННО. Почему-то Xmin очень плох, остальные хороши

In [32]:
for c in clfs:
    clf1 = Clf(votes.groupby('itemId')[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean(), votes.groupby('itemId')['Xmin_true'].mean(), c)
    clf2 = Clf(votes.groupby('itemId')[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean(), votes.groupby('itemId')['Ymin_true'].mean(), c) 
    clf3 = Clf(votes.groupby('itemId')[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean(), votes.groupby('itemId')['Xmax_true'].mean(), c)
    clf4 = Clf(votes.groupby('itemId')[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean(), votes.groupby('itemId')['Ymax_true'].mean(), c)
    
    data = votes.groupby('itemId')[['Xmin', 'Ymin', 'Xmax', 'Ymax']].mean()
    data['Xmin'] = clf1.predict(votes.groupby('itemId')[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())
    data['Ymin'] = clf2.predict(votes.groupby('itemId')[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())
    data['Xmax'] = clf3.predict(votes.groupby('itemId')[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())
    data['Ymax'] = clf4.predict(votes.groupby('itemId')[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())
    data = data.merge(answers, on=["itemId"])
    
    print('SCORE 2', clf2.score(votes.groupby('itemId')[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean(), data['Ymin_true']))
    print('SCORE 3', clf3.score(votes.groupby('itemId')[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean(), data['Xmax_true']))
    print('SCORE 4', clf4.score(votes.groupby('itemId')[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean(), data['Ymax_true']))
    
    data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true', 'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
    print(data["iou"].mean())

SCORE 2 0.6697204043324849
SCORE 3 0.5024364138174906
SCORE 4 0.9495297465775159
0.40517687308969236
SCORE 2 0.6965322251393553
SCORE 3 0.5889997196387734
SCORE 4 0.9645176524056386
0.4454263242632174


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


SCORE 2 0.7000994940192421
SCORE 3 0.5540531927584416
SCORE 4 0.9626279601451158
0.43320616947788465
SCORE 2 0.6796201253611487
SCORE 3 0.4833085954626075
SCORE 4 0.9453135884435006
0.4011614835162363
SCORE 2 0.6006124213017195
SCORE 3 0.3762588120325958
SCORE 4 0.957884822386143
0.37094980569394614


## Поправка -  Xmax тоже так себе...  Попробую ограничиться игреками

In [0]:
data['Xmin'] = clf1.predict(votes.groupby('itemId')[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())
data['Ymin'] = clf2.predict(votes.groupby('itemId')[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())
data['Xmax'] = clf3.predict(votes.groupby('itemId')[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())
data['Ymax'] = clf4.predict(votes.groupby('itemId')[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())

...сверимся...

In [0]:
data = data.merge(answers, on=["itemId"])

In [0]:
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)

In [0]:
data["iou"].mean()   # 0.5972182819504922  NOT BAD!!!

0.641741127661026

## TRUE MOMENT

In [0]:
votest = pd.read_csv("drive/My Drive/sna9/test1_data.csv", index_col=0)

In [0]:
votest.head()

,userId,itemId,Xmin,Ymin,Xmax,Ymax,L,H,P,S,L/H,D
0,1581,34804,86,640,242,743,156,103,518,16068,1.514563,186.935818
1,1351,34804,85,655,273,766,188,111,598,20868,1.693694,218.323155
2,161,34804,85,648,268,748,183,100,566,18300,1.830000,208.540164
3,313,5704,32,618,647,1268,615,650,2530,399750,0.946154,894.832387
4,1097,5704,25,620,602,1242,577,622,2398,358894,0.927653,848.417939


In [0]:
data = votest.groupby('itemId')[['Xmin', 'Ymin', 'Xmax', 'Ymax']].median()

In [0]:
#data['Xmin'] = clf1.predict(votest.groupby('itemId')[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())
Ymin_X = votes.groupby('itemId')[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean()
Ymin_Y = votes.groupby('itemId')['Ymin_true'].mean()
X_train, X_test, y_train, y_test = train_test_split(Ymin_X, Ymin_Y, test_size=0.2, random_state=42)
clf = BaggingRegressor(GradientBoostingRegressor()).fit(X_train, y_train)  
clf.score(X_test, y_test )
data['Ymin'] = clf.predict(votest.groupby('itemId')[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())
#data['Xmax'] = clf3.predict(votest.groupby('itemId')[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())
#data['Ymax'] = clf4.predict(votest.groupby('itemId')[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())

In [0]:
Xmax_X = votes.groupby('itemId')[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean()
Xmax_Y = votes.groupby('itemId')['Xmax_true'].mean()
X_train, X_test, y_train, y_test = train_test_split(Xmax_X, Xmax_Y, test_size=0.2, random_state=42)
clf = BaggingRegressor(GradientBoostingRegressor()).fit(X_train, y_train)  
clf.score(X_test, y_test )
data['Xmax'] = clf.predict(votest.groupby('itemId')[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())

In [0]:
Ymax_X = votes.groupby('itemId')[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean()
Ymax_Y = votes.groupby('itemId')['Ymax_true'].mean()
X_train, X_test, y_train, y_test = train_test_split(Ymax_X, Ymax_Y, test_size=0.2, random_state=42)
clf = BaggingRegressor(GradientBoostingRegressor()).fit(X_train, y_train)  
clf.score(X_test, y_test )
data['Ymax'] = clf.predict(votest.groupby('itemId')[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']].mean())

In [0]:
data.sort_index().to_csv("drive/My Drive/sna9/bag_xgb_woXmin.csv", header=None)